In [1]:
#import
import requests
import csv
import time
from datetime import date, timedelta
from twython import Twython
import math
from collections import namedtuple

#dates
t = date.today()
today = t.strftime("%d/%m")
y = t - timedelta(days=1)
yy = t - timedelta(days=2)
yesterday = y.strftime("%m%d%y")
yesterdayb = y.strftime("%d/%m")
twodaysago = yy.strftime("%m%d%y")
yesterdaymonth = y.strftime("%B")
yesterdayd = y.strftime('%d')
yesterdayday = ""
if yesterdayd == '01' or yesterdayd == '21' or yesterdayd == '31':
    yesterdayday = f"st"
elif yesterdayd == '02' or yesterdayd == '22':
    yesterdayday = f"nd"
elif yesterdayd == '03' or yesterdayd == '23':
    yesterdayday = f"rd"
else:
    yesterdayday = f"th"
yesterdayday

'th'

In [2]:
#artist & account selection (v2)
btsparam=(["BTS"],['3Nrfpe0tUJi4K4DXYWgMUX'],"BTS","MAP OF THE SOUL : 7","@BTS_twt","#BTS","#방탄소년단")
tsparam=(["Taylor Swift"],['06HL4z0CvFAxyc27GXpf02'],"TS","Lover","@taylorswift13","#TAYLORSWIFT",None)
bangswiftnation=("BANGSWIFTNATION",[btsparam,tsparam])
rmparam=(["RM"],['2auC28zjQyVTsiZKNgPRGs'],"RM","Strange","@BTS_twt","#RM","#방탄소년단RM #김남준 #BTS")
sugaparam=(["SUGA","Agust D"],['0ebNdVaOfp6N0oZ1guIxM8','5RmQ8k4l3HZ8JoPb4mNsML'],"SUGA","D-2","@BTS_twt",'#SUGA',"#방탄소년단슈가 #슈가 #BTS")
jhopeparam=(["j-hope"],['0b1sIQumIAsNbqAoIClSpy'],"JH","Chicken Noodle Soup (feat. Becky G)","@BTS_twt","#JHOPE","#방탄소년단제이홉 #제이홉 #BTS")
raplinechartdata=("raplinechartdata",[rmparam,sugaparam,jhopeparam])
accounts=[bangswiftnation,raplinechartdata]

In [3]:
#json
def databasepull(artisturi):
    jsons=[]
    for uri in artisturi:
        w=requests.get('http://localhost:8080/artistInfo?artistid='+uri)
        working=w.json()
        jsons.append(working)
    return jsons

#uri list loop
def urilistloop(jsons,artistname,newestrelease):
    masterlist = []
    appearsonlist = []
    newestreleaseuri=""
    for json in jsons:
        print("-")
        try:
            for album in json['data']['releases']['albums']['releases']:
                string=album['uri'].lstrip('spotify:album:')
                if album['name']==newestrelease:
                    newestreleaseuri="/album/"+string
                masterlist.append(string)
        except:
            print("No albums")
        try:
            for single in json['data']['releases']['singles']['releases']:
                string=single['uri'].lstrip('spotify:album:')
                if single['name']==newestrelease:
                    newestreleaseuri="/album/"+string
                masterlist.append(string)
        except:
            print("No singles")
    try:
        for feature in json['data']['releases']['appears_on']['releases']:
            string=feature['uri'].lstrip('spotify:album:')
            if feature['name']==newestrelease:
                newestreleaseuri="/album/"+string
            appearsonlist.append(string)
    except:
        print("No features")
    #manual eight add
    if artistname == ['SUGA','Agust D']:
        appearsonlist.append("5vJNAlQeTf9lsulO1YlmSt")
    return (masterlist, appearsonlist, newestreleaseuri)

#song & album counts loop
def songalbumcountsloop(masterlist,appearsonlist,newestreleaseuri,artistname,newestrelease):
    songplaycounts = {}
    albumplaycounts = {}
    for release in masterlist:
        query=f"http://localhost:8080/albumPlayCount?albumid={release}"
        req=requests.get(query)
        while req.status_code==500:
            print('Broken: '+release)
            time.sleep(5)
            req=requests.get(query)
        result=req.json()
        albumname=result['data']['name']
        albumplaycounts[albumname]=0
        result['data']['discs']
        for disc in result['data']['discs']:
            tracks=disc['tracks']
            for track in tracks:
                name=track['name']
                count=track['playcount']
                albumplaycounts[albumname]=albumplaycounts[albumname]+count
                if name in songplaycounts.keys():
                    if count == songplaycounts[name]:
                        continue
                    else:
                        name=f"{name} 2"
                songplaycounts[name]=count
    #appears on section
    for release in appearsonlist:
        query=f"http://localhost:8080/albumPlayCount?albumid={release}"
        req=requests.get(query)
        while req.status_code==500:
            print('Broken: '+release)
            time.sleep(5)
            req=requests.get(query)
        result=req.json()
        result['data']['discs']
        for disc in result['data']['discs']:
            tracks=disc['tracks']
            for track in tracks:
                artists=track['artists']
                for artist in artists:
                    name=artist['name']
                    for person in artistname:
                        if name == person:
                            title=track['name']
                            if title==newestrelease:
                                newestreleaseuri=track['uri'].lstrip('spotify:track:')
                                newestreleaseuri="/track/"+newestreleaseuri
                            count=track['playcount']
                            if title in songplaycounts.keys():
                                if count == songplaycounts[title]:
                                    continue
                            else:
                                name=f"{name} 2"
                                songplaycounts[title]=count
                        else:
                            continue
    return (songplaycounts,albumplaycounts,newestreleaseuri)

#count by creation, duplicate detection & removal
def countbyalbumfunc(albumplaycounts):
    countbyalbum = {}
    for name in albumplaycounts.keys():
        playcount=albumplaycounts[name]
        countbyalbum[playcount]=name
    return countbyalbum
def countbysongfunc(songplaycounts):
    countbysong = {}
    songstoremove = []
    for name in songplaycounts.keys():
        playcount=songplaycounts[name]
        try:
            if countbysong[playcount]==name:
                continue
            else:
                print(f"Found duplicate: {name}, {countbysong[playcount]}")
                songstoremove.append(name)
        except(KeyError):
            countbysong[playcount]=name
    for name in songstoremove:
        del songplaycounts[name]
    return countbysong

#previous day data call for day gain calculations
def twodaysagocall(account,shortening,twodaysago,albumorsong):
    twodaysagocounts = {}
    twodaysagocsv = f"/Users/ajkoenig/SPOTIFY/{account}/{shortening} {albumorsong} Counts/{twodaysago}{shortening}{albumorsong}PlayCounts.csv"
    with open(twodaysagocsv, mode='r') as infile:
        reader = csv.reader(infile)
        for row in reader:
            key=row[1]
            twodaysagocounts[key] = row[0]
    return twodaysagocounts

#total calculation, list & dict creation
def overalltotal(songplaycounts):
    total = sum(songplaycounts.values())
    return total

def overallchange(total,twodaysagosongplaycounts):
    overalldiff=0
    try:
        overalldiff = total-int(twodaysagosongplaycounts['Total'])
    except:
        overalldiff=0
    overalltot=(total,"Total")
    return (overalltot,overalldiff)

def daygains(countbyx,xplaycounts,twodaysagoxplaycounts=None):
    xs=[]
    xs=list(countbyx.items())
    if twodaysagoxplaycounts==None:
        return (xs)
    xdifference={}
    for key in xplaycounts.keys():
        try:
            change=int(xplaycounts[key])-int(twodaysagoxplaycounts[key])
        except:
            print(key)
            change=0
        xdifference[key]=change
    return(xs, xdifference)

#rapline playcounts (BTS)
def raplinesongs(songplaycounts):
    rapline=["Intro : Persona", "Jamais Vu", "Interlude : Shadow", "UGH!", "Respect", "Outro : Ego", "Dionysus", "Trivia 起 : Just Dance", "Trivia 轉 : Seesaw","Trivia 承 : Love","Her","Tear","Intro: Boy Meets Evil", "First Love", "Reflection", "MAMA","BTS Cypher 4","Intro: Never Mind","Intro: The Most Beautiful Moment in Life","Intro: What Am I to You","BTS Cypher Pt.3: Killer","Intro: Skool Luv Affair","BTS Cypher Pt.2: Triptych","Intro: O!RUL8,2?","Paldogangsan","BTS Cypher Pt.1","All Night (BTS World Original Soundtrack) [Pt. 3]","A Brand New Day (BTS World Original Soundtrack) [Pt. 2]"]
    raplineplaycounts={}
    for song in songplaycounts:
        if song in rapline:
            raplineplaycounts[song]=songplaycounts[song]
    return raplineplaycounts

#information (Play Count, Day Gain, Milestones)
def info(xplaycounts,xdifference):
    xcombined={}
    for x in xplaycounts:
        xcombined[x]={}
        currentxplaycount=xplaycounts[x]
        xcombined[x]["Play Count"]=currentxplaycount
        xcombined[x]["Day Gain"]=xdifference[x]
        if currentxplaycount > 100000000:
            nextmilestone=round(currentxplaycount,-8)
            if nextmilestone < currentxplaycount:
                nextmilestone=nextmilestone+100000000
        else:
            nextmilestone=round(currentxplaycount,-7)
            if nextmilestone < currentxplaycount:
                nextmilestone=nextmilestone+5000000
        xcombined[x]["Next Milestone"]=nextmilestone
        disttonextmilestone=nextmilestone-currentxplaycount
        xcombined[x]["Dist To Next Milestone"]=disttonextmilestone
        try:
            timetonextmilestone=disttonextmilestone/xdifference[x]
            xcombined[x]["Time To Next Milestone"]=timetonextmilestone
        except:
            xcombined[x]["Time To Next Milestone"]=0
        roundeddays=math.ceil(xcombined[x]["Time To Next Milestone"])
        try:
            dateofmilestone=timedelta(days=roundeddays)+y
            xcombined[x]["Predicted Date"]=dateofmilestone.strftime("%m%d%y")
        except:
            xcombined[x]["Predicted Date"]="Never"
    return xcombined

    #for x in sorted(xcombined.items(), key = lambda x: x[1][sortby]):
        #print(x[0]+"; "+'{:,}'.format(x[1]["Play Count"])+"; "+'{:,}'.format(x[1]["Day Gain"]))
        #print('{:,}'.format(x[1][sortby])+", "+x[0])

#print out (sortable)
def finalprint(dictionary,sortby):
    for entry in sorted(dictionary.items(), key = lambda x: x[1][sortby]):
        if sortby=="Time To Next Milestone":
            timetonext=entry[1]["Time To Next Milestone"]
            if timetonext < 30:
                nextmile=entry[1]["Next Milestone"]
                nextmile="{:,}".format(nextmile)
                print(entry[1]["Predicted Date"]+", "+nextmile+"; "+entry[0])
        else:
            continue
    return sorted(dictionary.items(), key = lambda x: x[1][sortby])

#tweet consolidation
def tweetconsolidation(newestrelease,albumplaycounts,albumdifference,songplaycounts,songdifference,total,overalldiff):
    try: 
        newestreleasecount="{:,}".format(albumplaycounts[newestrelease])
        newestreleasedaygain=albumdifference[newestrelease]
    except:
        newestreleasecount="{:,}".format(songplaycounts[newestrelease])
        newestreleasedaygain=songdifference[newestrelease]
    newestreleasedaygain = "{:,}".format(newestreleasedaygain)
    tweet_total = '{:,}'.format(total)
    overalldiffcomma="{:,}".format(overalldiff)
    return (newestreleasecount,newestreleasedaygain,tweet_total,overalldiffcomma)

#tweets
def tweetin(account,artistname,yesterdaymonth,yesterdayd,yesterdayday,newestrelease,twitterhandle,
            newestreleasecount,newestreleasedaygain,newestreleaseuri,tweet_total,overalldiffcomma,
            artisturi,mainhashtag,yesterdayb,brandhashtags):
    if account=='BANGSWIFTNATION':
        if int(yesterdayd) < 10:
            yesterdayd=yesterdayd.lstrip('0')
        yesterdayth=str(yesterdayd+yesterdayday)
        tweet_1=str("\U0001F4C8 | BxTS Spotify Update ("+yesterdaymonth+" "
        +yesterdayth+")\n\n"+newestrelease+" by "+twitterhandle+" - "
        +newestreleasecount+" streams (+"+newestreleasedaygain+")\n\nhttps://open.spotify.com"
        +newestreleaseuri)
        print(tweet_1)
        tweet_2=str("\U0001F4C8 | BxTS Spotify Update ("+yesterdaymonth+ " "+yesterdayth+")\n \n"+twitterhandle+" overall - "
        +tweet_total+" streams (+"+overalldiffcomma+")\n\nhttps://open.spotify.com/artist/"+artisturi[0])
        print(tweet_2)
    elif account=="raplinechartdata":
        tweet_2=None
        if artistname==["SUGA", "Agust D"]:
            tweet_1 = mainhashtag+" Spotify Update ("+yesterdayb+")\n\n"+newestrelease+" - "+newestreleasecount+" streams (+"+newestreleasedaygain+")\n\nOverall (SUGA & Agust D) - "+tweet_total+" streams (+"+overalldiffcomma+")\n\n"+brandhashtags+" "+twitterhandle+"\nhttps://open.spotify.com"+newestreleaseuri
            print(tweet_1)
        else:
            tweet_1 = mainhashtag+" Spotify Update ("+yesterdayb+")\n\n"+newestrelease+" - "+newestreleasecount+" streams (+" +newestreleasedaygain+")\n\nOverall - "+tweet_total+" streams (+"+overalldiffcomma+")\n\n"+brandhashtags+" "+twitterhandle+"\nhttps://open.spotify.com"+newestreleaseuri
            print(tweet_1)

    else:
        print("no account selected")
    return (tweet_1,tweet_2)

#csv write
def csvwrite(account,shortening,yesterday,albumorsong,albumorsongs):
    csv_file = f"/Users/ajkoenig/SPOTIFY/{account}/{shortening} {albumorsong} Counts/{yesterday}{shortening}{albumorsong}PlayCounts.csv"
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(albumorsongs)
    except IOError:
        print("I/O error: Counts")
    return
def tweet(tweet,account,consumer_key,consumer_secret,access_token,access_token_secret):
    twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret)
    twitter.update_status(status=tweet)
    print("Tweeted: %s" % tweet)
    return

In [4]:
def masterpull(artist):
    artistname=artist[0]
    artisturi=artist[1]
    shortening=artist[2]
    newestrelease=artist[3]
    twitterhandle=artist[4]
    mainhashtag=artist[5]
    brandhashtags=artist[6]
    
    jsons=databasepull(artisturi)

    urilistlooped=urilistloop(jsons,artistname,newestrelease)
    masterlist=urilistlooped[0]
    appearsonlist=urilistlooped[1]
    newestreleaseuri=urilistlooped[2]

    songalbumcounts=songalbumcountsloop(masterlist,appearsonlist,newestreleaseuri,artistname,newestrelease)
    songplaycounts=songalbumcounts[0]
    albumplaycounts=songalbumcounts[1]
    newestreleaseuri=songalbumcounts[2]

    countbyalbum=countbyalbumfunc(albumplaycounts)
    countbysong=countbysongfunc(songplaycounts)

    twodaysagoalbumplaycounts=twodaysagocall(account,shortening,twodaysago,"album")
    twodaysagosongplaycounts=twodaysagocall(account,shortening,twodaysago,"song")

    songdaygains=daygains(countbysong,songplaycounts,twodaysagosongplaycounts)
    songs=songdaygains[0]
    songdifference=songdaygains[1]
    albumdaygains=daygains(countbyalbum,albumplaycounts,twodaysagoalbumplaycounts)
    albums=albumdaygains[0]
    albumdifference=albumdaygains[1]
    total=overalltotal(songplaycounts)
    overalltot=overallchange(total,twodaysagosongplaycounts)[0]
    overalldiff=overallchange(total,twodaysagosongplaycounts)[1]
    songs.append(overalltot)

    albumscombined=info(albumplaycounts,albumdifference)
    songscombined=info(songplaycounts,songdifference)

    albumsortin=finalprint(albumscombined,"Time To Next Milestone")
    songsortin=finalprint(songscombined,"Time To Next Milestone")

    #tweetinfo=tweetconsolidation(newestrelease,albumplaycounts,albumdifference,songplaycounts,songdifference,total,overalldiff)
    #newestreleasecount=tweetinfo[0]
    #newestreleasedaygain=tweetinfo[1]
    #tweet_total=tweetinfo[2]
    #overalldiffcomma=tweetinfo[3]

    #tweets=tweetin(account,artistname,yesterdaymonth,yesterdayd,yesterdayday,newestrelease,twitterhandle,
                #newestreleasecount,newestreleasedaygain,newestreleaseuri,tweet_total,overalldiffcomma,
                #artisturi,mainhashtag,yesterdayb,brandhashtags)
    #tweet_1=tweets[0]
    #tweet_2=tweets[1]

    csvwrite(account,shortening,yesterday,"song",songs)
    csvwrite(account,shortening,yesterday,"album",albums)
    Final=namedtuple('Final', 'songssorted songs justsongs albumssorted albums songdiff songscombined')
    return Final(songsortin, songplaycounts, songs, albumsortin, albumplaycounts, songdifference, songscombined)

In [8]:
#timer
t = date.today()
today = t.strftime("%d/%m")
y = t - timedelta(days=1)
yy = t - timedelta(days=2)
yesterday = y.strftime("%m%d%y")
yesterdayb = y.strftime("%d/%m")
twodaysago = yy.strftime("%m%d%y")
yesterdaymonth = y.strftime("%B")
yesterdayd = y.strftime('%d')
yesterdayday = ""
if yesterdayd == '01' or yesterdayd == '21' or yesterdayd == '31':
    yesterdayday = f"st"
elif yesterdayd == '02' or yesterdayd == '22':
    yesterdayday = f"nd"
elif yesterdayd == '03' or yesterdayd == '23':
    yesterdayday = f"rd"
else:
    yesterdayday = f"th"
yesterdays=twodaysagocall("BANGSWIFTNATION","BTS",twodaysago,"song")
lastplaycount=int(yesterdays["Black Swan"])
query=f"http://localhost:8080/albumPlayCount?albumid=4RR8ROIGix5CbCGlXHA53I"
req=requests.get(query)
result=req.json()
currentplaycount=result['data']['discs'][0]['tracks'][0]['playcount']
while currentplaycount-lastplaycount == 0:
    print("Not changed yet")
    time.sleep(300)
    query=f"http://localhost:8080/albumPlayCount?albumid=4RR8ROIGix5CbCGlXHA53I"
    req=requests.get(query)
    while req.json()=="JSONDecodeError":
        time.sleep(5)
        req=requests.get(query)
        result=req.json()
        #FIX
    currentplaycount=result['data']['discs'][0]['tracks'][0]['playcount']
    print(currentplaycount)
if currentplaycount-lastplaycount < 0:
    print("Broken: negative")
else:
    #bangswiftnation
    
    #try:
    account="BANGSWIFTNATION"
    bts=masterpull(btsparam)
    #rapline=raplinesongs(bts.songs)
    #raplinediff=raplinesongs(bts.songdiff)
    #except:
        #print("BANGSWIFTNATION fail")
    #try:
        #account="raplinechartdata"
        #rm=masterpull(rmparam)
        #tweet(rm.tweet1,account,consumer_key,consumer_secret,access_token,access_token_secret)
        #suga=masterpull(sugaparam)
        #tweet(suga.tweet1,account,consumer_key,consumer_secret,access_token,access_token_secret)
        #jhope=masterpull(jhopeparam)
        #tweet(jhope.tweet1,account,consumer_key,consumer_secret,access_token,access_token_secret)
    #except:
        #print("raplinechartdata fail")

-
Found duplicate: Boy With Luv (feat. Halsey), Boy With Luv (Feat. Halsey)
Found duplicate: Dope - Chou Yabee! - - Japanese Ver., Dope - Chou Yabee! - Japanese Ver.
Found duplicate: Dream Glow (BTS World Original Soundtrack) [Pt. 1], Dream Glow (BTS World Original Soundtrack) - Pt. 1
Found duplicate: Best Of Me - Japanese Version, Best Of Me - Japanese ver.
Life Goes On
Fly To My Room
Blue & Grey
Skit
Telepathy
Dis-ease
Stay
Intro: Singularity
Outro: Tear
dimple
Outro: Her
Film out
Dynamite - Holiday Remix
A Song Make to You Smile
BE
Film out
Dynamite (Holiday Remix)
051721, 1,700,000,000; BE
051721, 60,000,000; Film out
051721, 15,000,000; Dynamite (Holiday Remix)
051821, 75,000,000; Youth
051821, 2,900,000,000; MAP OF THE SOUL : 7
051821, 80,000,000; MIC Drop/DNA/Crystal Snow
051821, 5,000,000; Danger (Japanese Ver.- / Standard Edition)
051821, 1,800,000,000; The Most Beautiful Moment in Life: Young Forever
051821, 60,000,000; SUGA's Interlude
051821, 1,800,000,000; Love Yourself 轉 

In [9]:
#songscombined
finalprint(bts.songscombined,"Day Gain")

[('Life Goes On',
  {'Play Count': 289548171,
   'Day Gain': 0,
   'Next Milestone': 300000000,
   'Dist To Next Milestone': 10451829,
   'Time To Next Milestone': 0,
   'Predicted Date': '051721'}),
 ('Fly To My Room',
  {'Play Count': 87494589,
   'Day Gain': 0,
   'Next Milestone': 90000000,
   'Dist To Next Milestone': 2505411,
   'Time To Next Milestone': 0,
   'Predicted Date': '051721'}),
 ('Blue & Grey',
  {'Play Count': 104343339,
   'Day Gain': 0,
   'Next Milestone': 200000000,
   'Dist To Next Milestone': 95656661,
   'Time To Next Milestone': 0,
   'Predicted Date': '051721'}),
 ('Skit',
  {'Play Count': 28203003,
   'Day Gain': 0,
   'Next Milestone': 30000000,
   'Dist To Next Milestone': 1796997,
   'Time To Next Milestone': 0,
   'Predicted Date': '051721'}),
 ('Telepathy',
  {'Play Count': 86853091,
   'Day Gain': 0,
   'Next Milestone': 90000000,
   'Dist To Next Milestone': 3146909,
   'Time To Next Milestone': 0,
   'Predicted Date': '051721'}),
 ('Dis-ease',
  {'P

In [13]:
account="BANGSWIFTNATION"
bts=masterpull(btsparam)
rapline=raplinesongs(bts.songs)
raplinediff=raplinesongs(bts.songdiff)

-
Found duplicate: Boy With Luv (feat. Halsey), Boy With Luv (Feat. Halsey)
Found duplicate: Intro: Singularity, Singularity
Found duplicate: Outro: Tear, Tear
Found duplicate: dimple, Dimple
Found duplicate: Outro: Her, Her
Found duplicate: Dope - Chou Yabee! - - Japanese Ver., Dope - Chou Yabee! - Japanese Ver.
Found duplicate: Dream Glow (BTS World Original Soundtrack) [Pt. 1], Dream Glow (BTS World Original Soundtrack) - Pt. 1
Found duplicate: Waste It On Me (feat. BTS), Waste It On Me
073120, 500,000,000; The Most Beautiful Moment in Life Pt.2
080220, 75,000,000; Lights/Boy With Luv
080220, 35,000,000; Stay Gold
080420, 300,000,000; Skool Luv Affair
080820, 3,100,000,000; Love Yourself 結 'Answer'
080820, 1,500,000,000; You Never Walk Alone
080920, 1,100,000,000; MAP OF THE SOUL : PERSONA
081320, 50,000,000; Youth
081320, 45,000,000; SUGA's Interlude
081620, 1,100,000,000; Wings
081920, 1,800,000,000; MAP OF THE SOUL : 7
082020, 1,100,000,000; Love Yourself 承 'Her'
082120, 40,000,0

In [5]:
def missedday(artist):
    artistname=artist[0]
    artisturi=artist[1]
    shortening=artist[2]
    newestrelease=artist[3]
    twitterhandle=artist[4]
    mainhashtag=artist[5]
    brandhashtags=artist[6]
    
    jsons=databasepull(artisturi)

    urilistlooped=urilistloop(jsons,artistname,newestrelease)
    masterlist=urilistlooped[0]
    appearsonlist=urilistlooped[1]
    newestreleaseuri=urilistlooped[2]

    songalbumcounts=songalbumcountsloop(masterlist,appearsonlist,newestreleaseuri,artistname,newestrelease)
    songplaycounts=songalbumcounts[0]
    albumplaycounts=songalbumcounts[1]
    newestreleaseuri=songalbumcounts[2]

    countbyalbum=countbyalbumfunc(albumplaycounts)
    countbysong=countbysongfunc(songplaycounts)

    songdaygains=daygains(countbysong,songplaycounts)
    songs=songdaygains
    albumdaygains=daygains(countbyalbum,albumplaycounts)
    albums=albumdaygains
    total=overalltotal(songplaycounts)
    overalltot=(total,'Total')
    songs.append(overalltot)

    csvwrite(account,shortening,yesterday,"song",songs)
    csvwrite(account,shortening,yesterday,"album",albums)
    return countbysong

In [6]:
account="BANGSWIFTNATION"
bts=missedday(btsparam)
#rapline=raplinesongs(bts.songs)
#raplinediff=raplinesongs(bts.songdiff)
#account="raplinechartdata"
#rm=missedday(rmparam)
##tweet(rm.tweet1,account,consumer_key,consumer_secret,access_token,access_token_secret)
#suga=missedday(sugaparam)
#tweet(suga.tweet1,account,consumer_key,consumer_secret,access_token,access_token_secret)
#jhope=missedday(jhopeparam)
#tweet(jhope.tweet1,account,consumer_key,consumer_secret,access_token,access_token_secret)

-
Found duplicate: Boy With Luv (feat. Halsey), Boy With Luv (Feat. Halsey)
Found duplicate: Intro: Singularity, Singularity
Found duplicate: Outro: Tear, Tear
Found duplicate: dimple, Dimple
Found duplicate: Outro: Her, Her
Found duplicate: Dope - Chou Yabee! - - Japanese Ver., Dope - Chou Yabee! - Japanese Ver.
Found duplicate: Dream Glow (BTS World Original Soundtrack) [Pt. 1], Dream Glow (BTS World Original Soundtrack) - Pt. 1
Found duplicate: Waste It On Me (feat. BTS), Waste It On Me


In [10]:
bts

Final(songssorted=[('Life Goes On', {'Play Count': 289548171, 'Day Gain': 0, 'Next Milestone': 300000000, 'Dist To Next Milestone': 10451829, 'Time To Next Milestone': 0, 'Predicted Date': '051721'}), ('Fly To My Room', {'Play Count': 87494589, 'Day Gain': 0, 'Next Milestone': 90000000, 'Dist To Next Milestone': 2505411, 'Time To Next Milestone': 0, 'Predicted Date': '051721'}), ('Blue & Grey', {'Play Count': 104343339, 'Day Gain': 0, 'Next Milestone': 200000000, 'Dist To Next Milestone': 95656661, 'Time To Next Milestone': 0, 'Predicted Date': '051721'}), ('Skit', {'Play Count': 28203003, 'Day Gain': 0, 'Next Milestone': 30000000, 'Dist To Next Milestone': 1796997, 'Time To Next Milestone': 0, 'Predicted Date': '051721'}), ('Telepathy', {'Play Count': 86853091, 'Day Gain': 0, 'Next Milestone': 90000000, 'Dist To Next Milestone': 3146909, 'Time To Next Milestone': 0, 'Predicted Date': '051721'}), ('Dis-ease', {'Play Count': 79002183, 'Day Gain': 0, 'Next Milestone': 80000000, 'Dist To 

In [ ]:
raplineinfo=info(rapline,raplinediff)
for song in rm.songssorted:
    raplineinfo[song[0]]=song[1]
for album in rm.albumssorted:
    raplineinfo[album[0]]=album[1]
for song in suga.songssorted:
    raplineinfo[song[0]]=song[1]
for album in suga.albumssorted:
    raplineinfo[album[0]]=album[1]
for song in jhope.songssorted:
    raplineinfo[song[0]]=song[1]
for album in jhope.albumssorted:
    raplineinfo[album[0]]=album[1]

In [ ]:
raplineinfo=finalprint(raplineinfo,"Time To Next Milestone")

In [ ]:
raplineinfo

In [ ]:
#milestones
def milestonetweetconsolidation(title,milestone,twitterhandle,platform,artist1,artist2=None,artist3=None):
    mainhashtag1=artist1[5]
    brandhashtags1=artist1[6]
    if artist2 != None:
        mainhashtag2=artist2[5]
        brandhashtags2=artist2[6]
    else:
        mainhashtag2=None
    if artist3 != None:
        mainhashtag3=artist3[5]
        brandhashtags3=artist3[6]
    else:
        mainhashtag3=None
    if platform =='Spotify':
        platformterminology=' streams on Spotify.'
    elif platform == 'YouTube':
        platformterminology=' views on YouTube.'
    else:
        print("Error: no platform")
        return
    if mainhashtag2==None and mainhashtag3==None:
        #milestone_tweet=str('.'+twitterhandle+"'s "+'"'+title+'" by '+mainhashtag1+' has surpassed '
                            #+milestone+platformterminology+" Congratulations!  \U0001F973 #"+title+milestone
                            #+"\n"+brandhashtags1)
        print(twitterhandle)
        milestone_tweet=str('"'+title+'" by '+twitterhandle+"'s "+mainhashtag1+' has surpassed '
                            +milestone+platformterminology+" Congratulations!  \U0001F973 #"+title+milestone
                            +"\n"+brandhashtags1)
        print(milestone_tweet)
    return milestone_tweet

In [ ]:
test=milestonetweetconsolidation('Jamais Vu','85M',jhopeparam[4],'Spotify',jhopeparam)
test

In [ ]:
def milestonetweetin(title,milestone,tweet,consumer_key,consumer_secret,access_token,access_token_secret):
    twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret)
    pathtofile=f"/Users/ajkoenig/Desktop/Upcoming Milestones/{title}{milestone}.png"
    photo = open(pathtofile, 'rb')
    response = twitter.upload_media(media=photo)
    twitter.update_status(status=tweet, media_ids=[response['media_id']])
    print("Tweeted: %s" % tweet)
    return

In [ ]:
#milestonetweetin("People","10M",test,consumer_key,consumer_secret,access_token,access_token_secret)

In [ ]:
rm.albumssorted

In [ ]:
account="BANGSWIFTNATION"
twodaysago= "071220"
bts = masterpull(btsparam)

In [ ]:
rapline

In [ ]:
flagemojisetup=requests.get('https://unpkg.com/country-flag-emoji-json@latest/json/flag-emojis.pretty.json')

In [ ]:
flagemojis=flagemojisetup.json()
flagemojis

In [ ]:
def countryemoji(country):
    for flag in flagemojis:
        if flag["name"]==country:
            emoji=flag["unicode"].split()
            emoji=emoji[0]
            emoji=emoji.replace("+","000")
    return emoji
def itunesnewpeak(title,country,countryemoji,total1s,link,twitterhandle,artists):
    if len(artists)==1:
        itunestweet=str(title+' by '+twitterhandle+"'s "+artists[0]+' has reached #1 on iTunes in '+country
                        +"! "+countryemoji+"\n" + "Total #1s: "+str(total1s)+" #"+title+str(total1s)+"Win \U0001F973 \n" +link)
    else:
        print('working')
    print(itunestweet)
    return itunestweet

In [ ]:
emoji=countryemoji("Peru")
itunesnewpeak("Outro: Tear", "Peru",emoji, 1, "whsoehwoiht", "@BTS_twt", ["#BTS"])